# LanceDB

## 1. Quick Start

### Installation

In [1]:
!pip install lancedb pandas sentence_transformers

### Connect to a database

In [2]:
import lancedb

#path
uri = "data/sample-lancedb"

db = lancedb.connect(uri)

/Users/miladmohammadi/Documents/Semester-3/TAs/IIR Workshop/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create a table from initial data

In [3]:
data = [
    {"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
    {"vector": [5.9, 26.5], "item": "bar", "price": 20.0},
    {"vector": [10, 13], "item": "fizz", "price":1}
]

tbl = db.create_table("my_table", data=data)

#### from pandas DataFrame

In [4]:
#If running local you may need to change the max recursion limit
import sys
sys.setrecursionlimit(10000)

import pandas as pd

df = pd.DataFrame(
    [
        {"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
        {"vector": [5.9, 26.5], "item": "bar", "price": 20.0},
        {"vector": [10, 13], "item": "fizz", "price":1}
    ]
)

tbl = db.create_table("my_table", data=df, mode="overwrite")

### Create an empty table

In [5]:
# Define Schema
from lancedb.pydantic import Vector, LanceModel

class DataSchema(LanceModel):
    vector: Vector(2)
    item: str
    price: float


tbl = db.create_table("empty_table", schema=DataSchema)

### Add data to a table

In [6]:
data = [
    {"vector": [1, 1], "item": "tar", "price": 100.0},
    {"vector": [7, 7], "item": "jar", "price": 200.0},
]

tbl.add(data)

In [7]:
tbl.to_pandas()

,vector,item,price
0,"[1.0, 1.0]",tar,100.0
1,"[7.0, 7.0]",jar,200.0


#### Add data in batch

In [8]:
df = pd.DataFrame(
    [
        {"vector": [12.34, 56.78], "item": "Chair", "price": 49.99},
        {"vector": [23.45, 67.89], "item": "Table", "price": 89.99},
        {"vector": [34.56, 78.90], "item": "Lamp", "price": 29.99},
        {"vector": [45.67, 89.01], "item": "Sofa", "price": 299.99},
        {"vector": [56.78, 90.12], "item": "Desk", "price": 149.99},
        {"vector": [67.89, 12.34], "item": "Bed", "price": 399.99},
        {"vector": [78.90, 23.45], "item": "Cabinet", "price": 199.99},
        {"vector": [89.01, 34.56], "item": "Bookshelf", "price": 59.99},
        {"vector": [90.12, 45.67], "item": "Monitor", "price": 129.99},
        {"vector": [15.23, 35.67], "item": "Keyboard", "price": 39.99},
        {"vector": [25.34, 45.78], "item": "Mouse", "price": 19.99},
        {"vector": [35.45, 55.89], "item": "Printer", "price": 89.99},
        {"vector": [45.56, 65.90], "item": "Scanner", "price": 59.99},
        {"vector": [55.67, 75.01], "item": "Router", "price": 49.99},
        {"vector": [65.78, 85.12], "item": "Switch", "price": 29.99},
        {"vector": [75.89, 95.23], "item": "Speaker", "price": 24.99},
        {"vector": [85.90, 12.34], "item": "Headphones", "price": 19.99},
        {"vector": [95.01, 22.45], "item": "Webcam", "price": 34.99},
        {"vector": [10.12, 32.56], "item": "Microphone", "price": 59.99},
        {"vector": [20.23, 42.67], "item": "Tablet", "price": 199.99},
        {"vector": [30.34, 52.78], "item": "Smartphone", "price": 699.99},
        {"vector": [40.45, 62.89], "item": "TV", "price": 499.99},
        {"vector": [50.56, 72.90], "item": "Blender", "price": 39.99},
        {"vector": [60.67, 82.01], "item": "Toaster", "price": 19.99},
        {"vector": [70.78, 92.12], "item": "Mixer", "price": 89.99},
        {"vector": [80.89, 10.23], "item": "Coffee Maker", "price": 59.99},
        {"vector": [90.90, 20.34], "item": "Air Conditioner", "price": 299.99},
        {"vector": [11.01, 30.45], "item": "Heater", "price": 49.99},
        {"vector": [21.12, 40.56], "item": "Fan", "price": 24.99},
        {"vector": [31.23, 50.67], "item": "Vacuum Cleaner", "price": 149.99},
        {"vector": [41.34, 60.78], "item": "Washing Machine", "price": 399.99},
        {"vector": [51.45, 70.89], "item": "Dryer", "price": 349.99},
        {"vector": [61.56, 80.90], "item": "Refrigerator", "price": 599.99},
        {"vector": [71.67, 90.01], "item": "Oven", "price": 299.99},
        {"vector": [81.78, 10.12], "item": "Dishwasher", "price": 249.99},
        {"vector": [91.89, 20.23], "item": "Microwave", "price": 99.99},
        {"vector": [13.14, 30.34], "item": "Electric Kettle", "price": 29.99},
        {"vector": [23.25, 40.45], "item": "Rice Cooker", "price": 39.99},
    ]
)

In [ ]:
'''It is recommended to use iterators to add large datasets in batches.
But for simplicity we used manually adding batches.'''

from tqdm import tqdm

batch_size = 10

start_index = 0

# Iterate over the DataFrame in chunks (batches)
for i in tqdm(range(start_index, len(df), batch_size)):
    # Slice the DataFrame for the current batch
    batch_df = df.iloc[i : i + batch_size]

    # Prepare a list to store the data in LanceDB-friendly format
    batch_data = []
    for _, row in batch_df.iterrows():
        data_dict = {
            "vector": row["vector"],
            "item": row["item"],
            "price": row["price"]
        }
        batch_data.append(data_dict)

    # Add the batch to your LanceDB table
    tbl.add(batch_data)

print("Data has been added to the table in batches.")

##### Using Iterators

In [9]:
def df_to_dict_batches(df: pd.DataFrame, batch_size: int = 128):
    """
    Yields data from a DataFrame in batches of dictionaries.
    Each batch is a list of dict, suitable for LanceDB ingestion.
    """
    for start_idx in range(0, len(df), batch_size):
        end_idx = start_idx + batch_size
        # Convert the batch of rows to a list of dict
        batch_dicts = df.iloc[start_idx:end_idx].to_dict(orient="records")
        yield batch_dicts

tbl = db.create_table(
    "batch_iterator_table",
    data=df_to_dict_batches(df, batch_size=10),  # <-- the iterator
    schema=DataSchema
)

In [10]:
tbl.to_pandas()

,vector,item,price
0,"[12.34, 56.78]",Chair,49.99
1,"[23.45, 67.89]",Table,89.99
2,"[34.56, 78.9]",Lamp,29.99
3,"[45.67, 89.01]",Sofa,299.99
4,"[56.78, 90.12]",Desk,149.99
5,"[67.89, 12.34]",Bed,399.99
6,"[78.9, 23.45]",Cabinet,199.99
7,"[89.01, 34.56]",Bookshelf,59.99
8,"[90.12, 45.67]",Monitor,129.99
9,"[15.23, 35.67]",Keyboard,39.99


### Open an existing table

In [11]:
print(db.table_names())

['batch_iterator_table', 'empty_table', 'my_table']


In [12]:
tbl = db.open_table("my_table")

### Search for nearest neighbors

In [13]:
tbl.search([3.3, 3.9]).limit(1).to_list()

[{'vector': [3.0999999046325684, 4.099999904632568],
  'item': 'foo',
  'price': 10.0,
  '_distance': 0.07999993860721588}]

#### Add metric for distance

In [14]:
#Metrics can be: default='l2', 'cosine', 'dot'
tbl.search([3.1, 20]).metric("cosine").limit(2).to_pandas()

,vector,item,price,_distance
0,"[5.9, 26.5]",bar,20.0,0.002131
1,"[3.1, 4.1]",foo,10.0,0.119375


In [15]:
tbl2 = db.open_table("empty_table")
tbl2.search([77, 7]).limit(5).to_pandas()

,vector,item,price,_distance
0,"[7.0, 7.0]",jar,200.0,4900.0
1,"[1.0, 1.0]",tar,100.0,5812.0


### Delete and Drop

#### Delete a row

In [16]:
tbl.to_pandas()

,vector,item,price
0,"[3.1, 4.1]",foo,10.0
1,"[5.9, 26.5]",bar,20.0
2,"[10.0, 13.0]",fizz,1.0


In [17]:
tbl.delete('item = "fizz"')
tbl.to_pandas()

,vector,item,price
0,"[3.1, 4.1]",foo,10.0
1,"[5.9, 26.5]",bar,20.0


#### Drop a table

In [18]:
db.drop_table("my_table")
db.drop_table("empty_table")

## 2. Working with Text and Embeddings

### Data

In [14]:
import pandas as pd

df = pd.DataFrame(
    [
        {
            "title": "Introduction to Neural Networks",
            "summary": "An overview of neural networks, their architecture, and basic functioning, including neurons, layers, and activation functions.",
            "difficulty": 2
        },
        {
            "title": "Transformers in NLP",
            "summary": "Explores the transformer architecture, its role in natural language processing tasks, and how it has revolutionized language models.",
            "difficulty": 4
        },
        {
            "title": "Reinforcement Learning Basics",
            "summary": "Covers the fundamental concepts of reinforcement learning, including agents, environments, rewards, and policy optimization.",
            "difficulty": 3
        },
        {
            "title": "Information Retrieval Systems",
            "summary": "Discusses the principles behind information retrieval systems, indexing, ranking algorithms, and evaluation metrics like precision and recall.",
            "difficulty": 3
        },
        {
            "title": "Convolutional Neural Networks Explained",
            "summary": "Delves into convolutional neural networks, their layers, convolution operations, pooling, and applications in image recognition.",
            "difficulty": 3
        },
        {
            "title": "BERT for Language Understanding",
            "summary": "An in-depth look at BERT (Bidirectional Encoder Representations from Transformers) and its applications in understanding context in language processing.",
            "difficulty": 4
        },
        {
            "title": "Semantic Search Techniques",
            "summary": "Explores advanced search techniques that understand the intent and contextual meaning of search queries to deliver more relevant results.",
            "difficulty": 4
        },
        {
            "title": "Support Vector Machines",
            "summary": "Introduces Support Vector Machines, their mathematical foundation, and applications in classification and regression tasks.",
            "difficulty": 3
        },
        {
            "title": "Generative Adversarial Networks",
            "summary": "Covers the concept of GANs, their architecture involving generators and discriminators, and their use in generating realistic data.",
            "difficulty": 5
        },
        {
            "title": "Word Embeddings in NLP",
            "summary": "Discusses word embeddings like Word2Vec and GloVe, their importance in capturing semantic relationships between words, and their applications.",
            "difficulty": 3
        },
        {
            "title": "Deep Reinforcement Learning",
            "summary": "Combines deep learning with reinforcement learning to solve complex tasks, highlighting algorithms like DQN and policy gradients.",
            "difficulty": 5
        },
        {
            "title": "Natural Language Generation",
            "summary": "Explores techniques for generating human-like text, including rule-based methods and modern neural approaches like GPT.",
            "difficulty": 4
        },
        {
            "title": "Attention Mechanisms in Deep Learning",
            "summary": "Details how attention mechanisms enhance neural network performance by allowing models to focus on relevant parts of the input.",
            "difficulty": 4
        },
        {
            "title": "Clustering Algorithms Overview",
            "summary": "Provides an overview of clustering algorithms such as K-Means, Hierarchical Clustering, and DBSCAN, and their use cases.",
            "difficulty": 2
        },
        {
            "title": "Sequence-to-Sequence Models",
            "summary": "Examines seq2seq models used in tasks like machine translation, including encoder-decoder architectures and attention.",
            "difficulty": 4
        },
        {
            "title": "Dimensionality Reduction Techniques",
            "summary": "Covers techniques like PCA and t-SNE for reducing the dimensionality of data while preserving essential structures.",
            "difficulty": 3
        },
        {
            "title": "Ethics in Artificial Intelligence",
            "summary": "Discusses the ethical considerations in AI development, including bias, fairness, transparency, and accountability.",
            "difficulty": 2
        },
        {
            "title": "Transfer Learning in Deep Learning",
            "summary": "Explores how transfer learning leverages pre-trained models to improve performance on related tasks with limited data.",
            "difficulty": 3
        },
        {
            "title": "Time Series Forecasting with LSTMs",
            "summary": "Details the use of Long Short-Term Memory networks for modeling and forecasting time-dependent data.",
            "difficulty": 4
        },
        {
            "title": "Graph Neural Networks",
            "summary": "Introduces graph neural networks and their applications in processing graph-structured data for tasks like node classification.",
            "difficulty": 5
        },
        {
            "title": "Hyperparameter Tuning Strategies",
            "summary": "Covers methods for optimizing hyperparameters in machine learning models, including grid search and Bayesian optimization.",
            "difficulty": 3
        },
        {
            "title": "Ensemble Methods in Machine Learning",
            "summary": "Explores ensemble techniques like Random Forests and Gradient Boosting that combine multiple models to improve accuracy.",
            "difficulty": 3
        },
        {
            "title": "Autoencoders for Data Compression",
            "summary": "Discusses autoencoders and their role in unsupervised learning for compressing and reconstructing data efficiently.",
            "difficulty": 4
        },
        {
            "title": "Recurrent Neural Networks Fundamentals",
            "summary": "Provides an introduction to RNNs, their structure, and how they handle sequential data with feedback connections.",
            "difficulty": 3
        },
        {
            "title": "Zero-Shot Learning Techniques",
            "summary": "Examines methods that enable models to recognize objects without having seen any training examples, using semantic information.",
            "difficulty": 5
        },
        {
            "title": "Machine Translation with Transformers",
            "summary": "Details how transformer models are applied to machine translation tasks, outperforming traditional RNN-based approaches.",
            "difficulty": 4
        },
        {
            "title": "Adversarial Machine Learning",
            "summary": "Explores how adversarial examples can deceive machine learning models and strategies to make models more robust.",
            "difficulty": 5
        },
        {
            "title": "Natural Language Understanding",
            "summary": "Covers the processes and techniques involved in enabling machines to comprehend and interpret human language.",
            "difficulty": 3
        },
        {
            "title": "Data Augmentation in Deep Learning",
            "summary": "Discusses techniques for artificially increasing the size of training datasets to improve model generalization.",
            "difficulty": 2
        },
        {
            "title": "Explainable AI (XAI)",
            "summary": "Delves into methods and tools that make AI models' decisions transparent and understandable to humans.",
            "difficulty": 4
        },
        {
            "title": "Federated Learning Concepts",
            "summary": "Introduces federated learning, where models are trained across multiple decentralized devices while keeping data localized.",
            "difficulty": 4
        },
        {
            "title": "Optimization Algorithms in Machine Learning",
            "summary": "Explores various optimization algorithms like SGD, Adam, and RMSprop used to train machine learning models effectively.",
            "difficulty": 3
        },
        {
            "title": "Multi-Modal Learning Approaches",
            "summary": "Covers techniques that integrate and process multiple types of data, such as text, images, and audio, within a single model.",
            "difficulty": 5
        },
    ]
)

df


,title,summary,difficulty
0,Introduction to Neural Networks,"An overview of neural networks, their architec...",2
1,Transformers in NLP,"Explores the transformer architecture, its rol...",4
2,Reinforcement Learning Basics,Covers the fundamental concepts of reinforceme...,3
3,Information Retrieval Systems,Discusses the principles behind information re...,3
4,Convolutional Neural Networks Explained,"Delves into convolutional neural networks, the...",3
5,BERT for Language Understanding,An in-depth look at BERT (Bidirectional Encode...,4
6,Semantic Search Techniques,Explores advanced search techniques that under...,4
7,Support Vector Machines,"Introduces Support Vector Machines, their math...",3
8,Generative Adversarial Networks,"Covers the concept of GANs, their architecture...",5
9,Word Embeddings in NLP,Discusses word embeddings like Word2Vec and Gl...,3


### Connect to DB

In [20]:
import lancedb
db = lancedb.connect(".lancedb")

### Define the embedding function

In [21]:
from lancedb.embeddings import get_registry
embedding_model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="mps")

### Define the data model or schema

In [22]:
#You should put HF_TOKEN in the Notebook enviroment variables

from lancedb.pydantic import LanceModel, Vector

class StudyNotes(LanceModel):
    title: str
    summary: str = embedding_model.SourceField()
    difficulty: int
    vector: Vector(embedding_model.ndims()) = embedding_model.VectorField()

### Create table and add data

In [23]:
def df_to_dict_batches(df: pd.DataFrame, batch_size: int = 128):
    """
    Yields data from a DataFrame in batches of dictionaries.
    Each batch is a list of dict, suitable for LanceDB ingestion.
    """
    for start_idx in range(0, len(df), batch_size):
        end_idx = start_idx + batch_size
        # Convert the batch of rows to a list of dict
        batch_dicts = df.iloc[start_idx:end_idx].to_dict(orient="records")
        yield batch_dicts

tbl = db.create_table(
    "cs_notes_table",
    data=df_to_dict_batches(df, batch_size=10),
    schema=StudyNotes
)

In [24]:
tbl.to_pandas()

,title,summary,difficulty,vector
0,Introduction to Neural Networks,"An overview of neural networks, their architec...",2,"[-0.019131705, -0.012864992, 0.02538711, -0.04..."
1,Transformers in NLP,"Explores the transformer architecture, its rol...",4,"[-0.03335983, 0.01862303, -0.00015819393, -0.0..."
2,Reinforcement Learning Basics,Covers the fundamental concepts of reinforceme...,3,"[-0.035786655, 0.04050146, 0.040627778, -0.021..."
3,Information Retrieval Systems,Discusses the principles behind information re...,3,"[-0.034227632, 0.03347169, 0.0036248541, -0.00..."
4,Convolutional Neural Networks Explained,"Delves into convolutional neural networks, the...",3,"[0.025591748, -0.044624545, -0.0327766, -0.035..."
5,BERT for Language Understanding,An in-depth look at BERT (Bidirectional Encode...,4,"[-0.048091616, -0.0010293722, -0.04051036, -0...."
6,Semantic Search Techniques,Explores advanced search techniques that under...,4,"[-0.05066493, 0.056105837, 0.013681952, -0.000..."
7,Support Vector Machines,"Introduces Support Vector Machines, their math...",3,"[-0.002767832, -0.029321738, -0.011279432, -0...."
8,Generative Adversarial Networks,"Covers the concept of GANs, their architecture...",5,"[-0.01153428, 0.01732661, -0.0022153237, -0.00..."
9,Word Embeddings in NLP,Discusses word embeddings like Word2Vec and Gl...,3,"[-0.014825601, -0.0048156823, 0.0042675766, -0..."


### Querying your table

In [25]:
query = "What does NLP stands for?"
#actual = table.search(query).limit(5).to_pydantic(Words)[0]
tbl.search(query).limit(5).to_pandas()


,title,summary,difficulty,vector,_distance
0,Transformers in NLP,"Explores the transformer architecture, its rol...",4,"[-0.03335983, 0.01862303, -0.00015819393, -0.0...",0.809757
1,BERT for Language Understanding,An in-depth look at BERT (Bidirectional Encode...,4,"[-0.048091616, -0.0010293722, -0.04051036, -0....",0.830706
2,Natural Language Understanding,Covers the processes and techniques involved i...,3,"[-0.008330684, 0.025997736, 0.036576882, -0.01...",0.852595
3,Machine Translation with Transformers,Details how transformer models are applied to ...,4,"[-0.048571706, 0.016223863, 0.012704103, -0.01...",0.889645
4,Word Embeddings in NLP,Discusses word embeddings like Word2Vec and Gl...,3,"[-0.014825601, -0.0048156823, 0.0042675766, -0...",0.890143


#### Hybrid Search

In [26]:
tbl.create_fts_index('summary', use_tantivy=False)
tbl.search(query, query_type="hybrid").limit(5).to_pandas()

,title,summary,difficulty,vector,_relevance_score
0,Transformers in NLP,"Explores the transformer architecture, its rol...",4,"[-0.03335983, 0.01862303, -0.00015819393, -0.0...",0.016393
1,Data Augmentation in Deep Learning,Discusses techniques for artificially increasi...,2,"[0.00042642147, 0.03388558, 0.06320969, -0.047...",0.016393
2,BERT for Language Understanding,An in-depth look at BERT (Bidirectional Encode...,4,"[-0.048091616, -0.0010293722, -0.04051036, -0....",0.016129
3,Autoencoders for Data Compression,Discusses autoencoders and their role in unsup...,4,"[-0.0008582097, 0.0154731665, -0.019677423, -0...",0.016129
4,Natural Language Understanding,Covers the processes and techniques involved i...,3,"[-0.008330684, 0.025997736, 0.036576882, -0.01...",0.015873


# Working with LLMs

## LLMs API providers Clients

### Install OpenAI client

In [1]:
!pip install openai

### Make a client object

In [2]:
GROK_BASE_URL = "https://api.x.ai/v1"

#If you are using Google Colab save in your Keys and then
'''from google.colab import userdata
GROK_API_KEY = userdata.get('GROK_API_KEY')'''

#If running locally save in your environment variables (based on OS)
import os
GROK_API_KEY = os.getenv("GROK_API_KEY")
GROK_MODEL_NAME = "grok-2-1212"

from openai import OpenAI
grok_client = OpenAI(api_key=GROK_API_KEY, base_url=GROK_BASE_URL)

### Hello World

In [3]:
response = grok_client.chat.completions.create(
  model=GROK_MODEL_NAME,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello World!"},
  ],
  temperature = 0.3
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
response

ChatCompletion(id='62b58c8d-d83d-4ae7-b179-6a71b50cde07', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735479157, model='grok-2-1212', object='chat.completion', service_tier=None, system_fingerprint='fp_149a247ffa', usage=CompletionUsage(completion_tokens=10, prompt_tokens=19, total_tokens=29, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, text_tokens=19, image_tokens=0)))

### Structured Output

In [7]:
user_math_problem = "6+4*5-1"

response = grok_client.chat.completions.create(
  model=GROK_MODEL_NAME,
  messages=[
    {"role": "system",
     "content": "You are a helpful assistant."},
    {"role": "user",
     "content": "Solve {math_problem} step by step".format(math_problem = user_math_problem)},
  ],
  temperature = 0.4
)
print(response.choices[0].message.content)

To solve the expression \(6 + 4 \times 5 - 1\) step by step, we need to follow the order of operations, often remembered by the acronym PEMDAS (Parentheses, Exponents, Multiplication and Division (from left to right), Addition and Subtraction (from left to right)).

Let's break it down:

1. **Identify and perform any multiplication or division first:**
   The expression contains one multiplication: \(4 \times 5\).
   \[
   4 \times 5 = 20
   \]
   So, the expression now becomes:
   \[
   6 + 20 - 1
   \]

2. **Next, perform any addition or subtraction from left to right:**
   First, we do the addition:
   \[
   6 + 20 = 26
   \]
   Now the expression is:
   \[
   26 - 1
   \]
   Finally, we do the subtraction:
   \[
   26 - 1 = 25
   \]

So, the final answer is:
\[
25
\]


#### Create Pydantic as Output Schema

In [8]:
from pydantic import BaseModel, Field

class MathSolution(BaseModel):
    steps: str = Field(description="Thinking steps to solve the math problem.")
    final_answer: int = Field(description="Final answer of the math problem.")

In [9]:
response = grok_client.beta.chat.completions.parse(
    model=GROK_MODEL_NAME,
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant."},
      {"role": "user",
      "content": "Solve {math_problem} step by step".format(math_problem = user_math_problem)},
    ],
    response_format =MathSolution,
    temperature=0.4
    )

response.choices[0].message.parsed

MathSolution(steps='1. Start with the expression: 6 + 4 * 5 - 1\n2. According to the order of operations (PEMDAS/BODMAS), multiplication comes before addition and subtraction. So, we first calculate the multiplication: 4 * 5 = 20\n3. Now the expression is: 6 + 20 - 1\n4. Next, we perform the addition: 6 + 20 = 26\n5. Finally, we perform the subtraction: 26 - 1 = 25\n\nThe final answer is 25.', final_answer=25)

In [10]:
response.choices[0].message.parsed.final_answer

25

In [11]:
from pydantic import BaseModel, Field
from typing import List

class MathSolution2(BaseModel):
    steps: List[str] = Field(description="Thinking steps to solve the math problem.")
    final_answer: int = Field(description="Final answer of the math problem.")

response = grok_client.beta.chat.completions.parse(
    model=GROK_MODEL_NAME,
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant."},
      {"role": "user",
      "content": "Solve {math_problem} step by step".format(math_problem = user_math_problem)},
    ],
    response_format = MathSolution2,
    )

response.choices[0].message.parsed

MathSolution2(steps=['1. **Identify and apply the order of operations**: According to the order of operations (PEMDAS/BODMAS), you should perform multiplication and division before addition and subtraction.', '2. **Perform the multiplication**: In the expression `6 + 4 * 5 - 1`, the multiplication is `4 * 5`.', '   - Calculation: `4 * 5 = 20`', '3. **Rewrite the expression with the result of multiplication**: The expression now becomes `6 + 20 - 1`.', '4. **Perform the addition**: Next, you add `6` and `20`.', '   - Calculation: `6 + 20 = 26`', '5. **Rewrite the expression with the result of addition**: The expression now becomes `26 - 1`.', '6. **Perform the subtraction**: Finally, you subtract `1` from `26`.', '   - Calculation: `26 - 1 = 25`', '7. **Final answer**: The result of `6 + 4 * 5 - 1` is `25`.'], final_answer=25)

In [12]:
response.choices[0].message.parsed.final_answer

25

### RAG

#### Data

#### Search to bring context (a fake Search!)

In [15]:
def get_random_summaries(dataframe: pd.DataFrame, n: int = 5) -> list:
    #Randomly chooses 'n' rows from the given DataFrame and returns a list of their summaries.
    random_rows = dataframe.sample(n)
    summaries_list = random_rows["summary"].tolist()
    return summaries_list

random_summaries = get_random_summaries(df, n=5)
print("Selected 5 random summaries:")
for s in random_summaries:
    print("-", s)

Selected 5 random summaries:
- Details how transformer models are applied to machine translation tasks, outperforming traditional RNN-based approaches.
- Provides an introduction to RNNs, their structure, and how they handle sequential data with feedback connections.
- Explores how transfer learning leverages pre-trained models to improve performance on related tasks with limited data.
- Examines seq2seq models used in tasks like machine translation, including encoder-decoder architectures and attention.
- Discusses word embeddings like Word2Vec and GloVe, their importance in capturing semantic relationships between words, and their applications.


#### Build System and User Prompts

In [16]:
user_query = "What does NLP stand for?"

def build_prompts(user_query: str, context_summaries: list) -> tuple:
    system_prompt = ""
    system_prompt += "You are a helpful assistant."
    system_prompt += "Answer user query using the provided context, if context is not related tell user."
    system_prompt += "You should always answer user question, if context is not related answer by your own knowledge."



    user_prompt = ""
    user_prompt += f"User question: {user_query}\n"
    user_prompt += "Context:\n"
    for i, summary in enumerate(context_summaries, start=1):
        user_prompt += f"{i}. {summary}\n"

    user_prompt += "End of context."
    return system_prompt, user_prompt

# Example usage:
system_prompt, user_prompt = build_prompts(user_query, random_summaries)
print("\nSystem Prompt:")
print(system_prompt)
print("\nUser Prompt:")
print(user_prompt)


System Prompt:
You are a helpful assistant.Answer user query using the provided context, if context is not related tell user.You should always answer user question, if context is not related answer by your own knowledge.

User Prompt:
User question: What does NLP stand for?
Context:
1. Details how transformer models are applied to machine translation tasks, outperforming traditional RNN-based approaches.
2. Provides an introduction to RNNs, their structure, and how they handle sequential data with feedback connections.
3. Explores how transfer learning leverages pre-trained models to improve performance on related tasks with limited data.
4. Examines seq2seq models used in tasks like machine translation, including encoder-decoder architectures and attention.
5. Discusses word embeddings like Word2Vec and GloVe, their importance in capturing semantic relationships between words, and their applications.
End of context.


#### RAG

In [17]:
response = grok_client.chat.completions.create(
    model=GROK_MODEL_NAME,
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
    ],
    temperature = 0.2
    )

response.choices[0].message.content

"NLP stands for Natural Language Processing. This context does not directly relate to the definition of NLP, so I've answered based on my own knowledge."

#### RAG with Structured Output

In [18]:
def build_prompts(user_query: str, context_summaries: list) -> tuple:
    system_prompt = ""
    system_prompt += "You are a helpful assistant."
    system_prompt += "If the context is related to user query return True, else False."

    user_prompt = ""
    user_prompt += f"User question: {user_query}\n"
    user_prompt += "Context:\n"
    for i, summary in enumerate(context_summaries, start=1):
        user_prompt += f"{i}. {summary}\n"

    user_prompt += "End of context."
    return system_prompt, user_prompt

# Example usage:
system_prompt, user_prompt = build_prompts(user_query, random_summaries)
print("\nSystem Prompt:")
print(system_prompt)
print("\nUser Prompt:")
print(user_prompt)


System Prompt:
You are a helpful assistant.If the context is related to user query return True, else False.

User Prompt:
User question: What does NLP stand for?
Context:
1. Details how transformer models are applied to machine translation tasks, outperforming traditional RNN-based approaches.
2. Provides an introduction to RNNs, their structure, and how they handle sequential data with feedback connections.
3. Explores how transfer learning leverages pre-trained models to improve performance on related tasks with limited data.
4. Examines seq2seq models used in tasks like machine translation, including encoder-decoder architectures and attention.
5. Discusses word embeddings like Word2Vec and GloVe, their importance in capturing semantic relationships between words, and their applications.
End of context.


In [19]:
class RelevancyCheck(BaseModel):
    explanation: str = Field(description="Reasoning")
    is_relevant: bool

response = grok_client.beta.chat.completions.parse(
    model=GROK_MODEL_NAME,
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
    ],
    response_format = RelevancyCheck,
    )

response.choices[0].message.parsed

RelevancyCheck(explanation="The context provided focuses on specific aspects of natural language processing (NLP) such as transformer models, RNNs, transfer learning, seq2seq models, and word embeddings. However, it does not directly define what NLP stands for. The user's query specifically asks for the meaning of the acronym NLP, which is not explicitly answered in any of the context points.", is_relevant=False)

## Langchain

### Install Langchain

In [32]:
!pip install -qU langchain langchain-openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Create a ChatModel client

In [35]:
PROVIDER_BASE_URL = "https://api.together.xyz/v1"
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url=GROK_BASE_URL,
    api_key=os.getenv("GROK_API_KEY"),
    model=GROK_MODEL_NAME,
)

### A simple LLM call

In [36]:
model.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'text_tokens': 10, 'image_tokens': 0}}, 'model_name': 'grok-2-1212', 'system_fingerprint': 'fp_149a247ffa', 'finish_reason': 'stop', 'logprobs': None}, id='run-056d33ad-1a8d-4870-a97c-d34b63522f2d-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {}})

In [37]:
print(model.invoke("Who are you?").content)

I am Grok, created by xAI. I'm an AI designed to provide helpful and truthful answers, often with a dash of outside perspective on humanity. Think of me as a friendly, cosmic guide, here to assist and enlighten. What's on your mind?


### Structured Output using Langchain

In [38]:
from pydantic import BaseModel, Field
from typing import List

class Greeting(BaseModel):
    sentence: str
    possible_negative_interpretation: str

model_hello_world = model.with_structured_output(Greeting)
model_hello_world

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x32e886a90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x32cccf950>, root_client=<openai.OpenAI object at 0x32c036f50>, root_async_client=<openai.AsyncOpenAI object at 0x32c6ff910>, model_name='grok-2-1212', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.x.ai/v1'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Greeting', 'description': '', 'parameters': {'properties': {'sentence': {'type': 'string'}, 'possible_negative_interpretation': {'type': 'string'}}, 'required': ['sentence', 'possible_negative_interpretation'], 'type': 'object'}}}], 'parallel_tool_calls': False, 'tool_choice': {'type': 'function', 'function': {'name': 'Greeting'}}}, config={}, config_factories=[])
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.Greeting'>])

In [39]:
model_hello_world.invoke("Hello how are you")

Greeting(sentence='Hello how are you', possible_negative_interpretation='None')

### Chain of Thought using Structured Output

In [41]:
from pydantic import BaseModel, Field
from typing import List

class Step(BaseModel):
    explanation: str
    output: str = Field("rewrote question")

class MathReasoning(BaseModel):
    steps: List[Step] = Field(description="Reasoning Steps to get to final Result")
    final_answer: int = Field(description="Final Result of math expression.")
    
math_model = model.with_structured_output(MathReasoning)
math_model


RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x32e886a90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x32cccf950>, root_client=<openai.OpenAI object at 0x32c036f50>, root_async_client=<openai.AsyncOpenAI object at 0x32c6ff910>, model_name='grok-2-1212', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.x.ai/v1'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MathReasoning', 'description': '', 'parameters': {'properties': {'steps': {'description': 'Reasoning Steps to get to final Result', 'items': {'properties': {'explanation': {'type': 'string'}, 'output': {'default': 'rewrote question', 'type': 'string'}}, 'required': ['explanation'], 'type': 'object'}, 'type': 'array'}, 'final_answer': {'description': 'Final Result of math expression.', 'type': 'integer'}}, 'required': ['steps', 'final_answer'], 'type': 'object'}}}], 'parallel_tool_calls': False, 'to

In [42]:
from langchain_core.prompts import ChatPromptTemplate

math_system_template = "Calculate the following math expression step by step."

math_prompt_template = ChatPromptTemplate.from_messages(
    [("system", math_system_template), ("user", "This is the problem: {expression}")]
)
math_prompt_template

ChatPromptTemplate(input_variables=['expression'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Calculate the following math expression step by step.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['expression'], input_types={}, partial_variables={}, template='This is the problem: {expression}'), additional_kwargs={})])

In [43]:
math_test_prompt = math_prompt_template.invoke({"expression": "25 * 2 + 10"})
math_test_prompt

ChatPromptValue(messages=[SystemMessage(content='Calculate the following math expression step by step.', additional_kwargs={}, response_metadata={}), HumanMessage(content='This is the problem: 25 * 2 + 10', additional_kwargs={}, response_metadata={})])

In [44]:
print(math_model.invoke(math_test_prompt))

steps=[Step(explanation='First, we need to perform the multiplication operation. We multiply 25 by 2.', output='50'), Step(explanation='Next, we add 10 to the result of the multiplication.', output='60')] final_answer=60


In [45]:
math_chain = math_prompt_template | math_model
math_results = math_chain.invoke({"expression": "25 * 2 + 10"})
math_results

MathReasoning(steps=[Step(explanation='First, we need to multiply 25 by 2. This gives us 50.', output='50'), Step(explanation='Next, we add 10 to the result of the multiplication. 50 + 10 equals 60.', output='60')], final_answer=60)

In [46]:
math_results.final_answer

60

### RAG + Structured Output

In [47]:
class QueryGenerator(BaseModel):
    queries: List[str] = Field(description="List of detailed questions that can be answered using the provided summaries")

rag_model = model.with_structured_output(QueryGenerator)

In [48]:
rag_prompt_template = ChatPromptTemplate.from_template("""You are given a context and a question.
Your task is to generate 3 specific user queries similar but with diversity of user question for context search that can be answered using these context.
Context: {context}
User Question: {question}
""")

rag_chain = rag_prompt_template | rag_model
rag_chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are given a context and a question.\nYour task is to generate 3 specific user queries similar but with diversity of user question for context search that can be answered using these context.\nContext: {context}\nUser Question: {question}\n'), additional_kwargs={})])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x32e886a90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x32cccf950>, root_client=<openai.OpenAI object at 0x32c036f50>, root_async_client=<openai.AsyncOpenAI object at 0x32c6ff910>, model_name='grok-2-1212', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.x.ai/v1'), kwargs={'tools': [{'type': 'function', 'funct

In [49]:
context = "Examines the applications of deep learning techniques in the healthcare industry, including diagnostics and personalized medicine."
question = "What are the applications of deep learning in healthcare?"

rag_chain.invoke({"context": context, "question": question})

QueryGenerator(queries=['How is deep learning used for diagnostics in healthcare?', 'What role does deep learning play in personalized medicine?', 'Can you list specific deep learning applications in the healthcare sector?'])